In [1]:
using Plots
using Printf

In [58]:
f(x, y) = 3x^2 + 2y^2
h(x, y) = 1 - x - y
dL(x, c) = [6x[1] - c / (x[1] + x[2] - 1), 4x[2] - c / (x[1] + x[2] - 1)]

dL (generic function with 1 method)

In [99]:
function BFGS(x, c, epsilon)
    Hinv = [1 0; 0 1]
    xold = copy(x)
    xnew = xold - 0.01 .* Hinv * dL(xold, c)
    while sum((xnew - xold).^2) > epsilon
        s = xnew - xold
        t = dL(xnew, c) - dL(xold, c)
        xold = copy(xnew)
        Hinv = (
            Hinv + (s' * t + t' * Hinv * t) * s * s' ./ (s' * t)^2 -
            (Hinv * t * s' + s * t' * Hinv) ./ (s' * t)
        )
        xnew = xold - 0.01 .* Hinv * dL(xold, c)
    end
    xnew
end

BFGS (generic function with 2 methods)

## BFGSの正当性チェック

In [23]:
df(x) = [10x[1], 2x[2]]
pos = 10 .* rand(2) .- 5

2-element Array{Float64,1}:
  4.900918435160682 
 -0.6688083986312998

In [24]:
function BFGS(x, epsilon)
    Hinv = [1 0; 0 1]
    xold = copy(x)
    xnew = xold - Hinv * df(x)
    while sqrt(sum((xnew - xold).^2)) > epsilon
        s = xnew - xold
        t = df(xnew) - df(xold)
        xold = copy(xnew)
        Hinv = (
            Hinv + (s' * t + t' * Hinv * t) * s * s' ./ (s' * t)^2 -
            (Hinv * t * s' + s * t' * Hinv) ./ (s' * t)
        )
        xnew = xold - Hinv * df(xold)
    end
    xnew
end

BFGS (generic function with 2 methods)

In [25]:
BFGS(pos, 1e-6)

2-element Array{Float64,1}:
  1.1287416632089007e-12
 -3.4080913071973092e-15

## Barrier Method

In [189]:
function barrier(x, eps_barrier, eps_bfgs)
    xtrail = []
    ytrail = []
    append!(xtrail, x[1])
    append!(ytrail, x[2])

    xold = copy(x)
    xnew = BFGS(xold, 100.0, eps_bfgs)
    append!(xtrail, xnew[1])
    append!(ytrail, xnew[2])
    c = 1.0
    while sum((xnew - xold).^2) > eps_barrier
        c += 1.0
        xold = copy(xnew)
        xnew = BFGS(xold, 100. / c, eps_bfgs)
        append!(xtrail, xnew[1])
        append!(ytrail, xnew[2])
    end
    xtrail, ytrail
end

barrier (generic function with 1 method)

In [199]:
eps_barrier = 1e-10
eps_bfgs = 1e-5
x = [1., 1.]

2-element Array{Float64,1}:
 1.0
 1.0

In [200]:
xtrail, ytrail = barrier(x, eps_barrier, eps_bfgs)

(Any[1.0, 2.64922, 2.20728, 1.86234, 1.6594, 1.52009, 1.41768, 1.33735, 1.27219, 1.21851  …  0.409621, 0.409615, 0.40961, 0.409604, 0.409599, 0.409593, 0.409588, 0.409582, 0.409577, 0.409571], Any[1.0, 3.89031, 3.29704, 2.80132, 2.50597, 2.303, 2.15461, 2.03907, 1.9463, 1.87102  …  0.614503, 0.614495, 0.614486, 0.614478, 0.614469, 0.614461, 0.614453, 0.614444, 0.614436, 0.614428])

In [212]:
x = y = range(-4.0, 4.0, length=1000)
p = plot(x, y, f, st=[:contourf])
for i in 1:length(xtrail)-1
    plot!(p[1], [xtrail[i], xtrail[i+1]], [ytrail[i], ytrail[i+1]], line=(:white, 1), legend=:none)
end
posx = @sprintf "%.2f" xtrail[end]
posy = @sprintf "%.2f" ytrail[end]
annotate!(xtrail[end], ytrail[end], text("($(posx), $(posy))", 10, :center, :white))
png("barrier")

In [208]:
@sprintf "%.2f" ytrail[end]

"0.61"

## Penalty Method

In [2]:
function BFGS(x, epsilon, f, c...)
    Hinv = [1 0; 0 1]
    xold = copy(x)
    df = ifelse(length(c) == 0, f(xold, 0), f(xold, c[1]))
    xnew = xold - 0.01 * Hinv * df
    println(xnew)
    while sum((xnew - xold).^2) > epsilon
        s = xnew - xold
        t = ifelse(length(c) == 0, f(xnew, 0) - f(xold, 0), f(xnew, c[1]) - f(xold, c[1]))
        xold = copy(xnew)
        Hinv = (
            Hinv + (s' * t + t' * Hinv * t) * s * s' ./ (s' * t)^2 -
            (Hinv * t * s' + s * t' * Hinv) ./ (s' * t)
        )
        df = ifelse(length(c) == 0, f(xold, 0), f(xold, c[1]))
        xnew = xold - 0.01 * Hinv * df
        println(xnew)
    end
    xnew
end

BFGS (generic function with 1 method)

In [3]:
h(x, y) = 1 - x - y
nablaL(x, c) = [
    6x[1] + c * ifelse(max(h(x[1], x[2]), 0) == 0, 0, -2 * h(x[1], x[2])),
    4x[2] + c * ifelse(max(h(x[1], x[2]), 0) == 0, 0, -2 * h(x[1], x[2]))
]

nablaL (generic function with 1 method)

In [7]:
function penalty(x, eps_penalty, eps_bfgs)
    xold = copy(x)
    c = 0.1
    xnew = BFGS(x, eps_bfgs, nablaL, c)
    println("--------------------------------")
    while sum((xnew - xold).^2) > eps_penalty
        c *= 1.05
        xold = copy(xnew)
        xnew = BFGS(xold, eps_bfgs, nablaL, c)
        println("--------------------------------")
    end
    xnew
end

penalty (generic function with 1 method)

In [8]:
eps_penalty = 1e-8
eps_bfgs = 1e-5
x = [1., 1.]

2-element Array{Float64,1}:
 1.0
 1.0

In [9]:
penalty(x, eps_penalty, eps_bfgs)

[0.94, 0.96]
[0.934551, 0.94151]
[0.913964, 0.937064]
[0.905319, 0.924258]
[0.895802, 0.915485]
[0.886936, 0.90618]
[0.878041, 0.897155]
[0.869267, 0.888174]
[0.860573, 0.879295]
[0.851967, 0.870501]
[0.843448, 0.861796]
[0.835013, 0.853178]
[0.826663, 0.844647]
[0.818396, 0.8362]
[0.810212, 0.827838]
[0.80211, 0.81956]
[0.794089, 0.811364]
[0.786148, 0.803251]
[0.778287, 0.795218]
[0.770504, 0.787266]
[0.762799, 0.779393]
[0.755171, 0.771599]
[0.747619, 0.763883]
[0.740143, 0.756244]
[0.732742, 0.748682]
[0.725414, 0.741195]
[0.71816, 0.733783]
[0.710978, 0.726445]
[0.703869, 0.719181]
[0.69683, 0.711989]
[0.689862, 0.704869]
[0.682963, 0.697821]
[0.676133, 0.690842]
[0.669372, 0.683934]
[0.662678, 0.677095]
[0.656052, 0.670324]
[0.649491, 0.66362]
[0.642996, 0.656984]
[0.636566, 0.650414]
[0.630201, 0.64391]
[0.623899, 0.637471]
[0.61766, 0.631096]
[0.611483, 0.624785]
[0.605368, 0.618538]
[0.599314, 0.612352]
[0.593321, 0.606229]
[0.587388, 0.600166]
[0.581514, 0.594165]
[0.575699, 

Excessive output truncated after 524292 bytes.

9.01963]
[8.19391, 8.92943]
[8.11197, 8.84014]
[8.03085, 8.75174]
[7.95054, 8.66422]
[7.87104, 8.57758]
[7.79233, 8.4918]
[7.7144, 8.40688]
[7.63726, 8.32281]
[7.56089, 8.23959]
[7.48528, 8.15719]
[7.41042, 8.07562]
[7.33632, 7.99486]
[7.26296, 7.91491]
[7.19033, 7.83576]
[7.11842, 7.75741]
[7.04724, 7.67983]
[6.97677, 7.60303]
[6.907, 7.527]
[6.83793, 7.45173]
[6.76955, 7.37722]
[6.70186, 7.30344]
[6.63484, 7.23041]
[6.56849, 7.15811]
[6.5028, 7.08652]
[6.43778, 7.01566]
[6.3734, 6.9455]
[6.30966, 6.87605]
[6.24657, 6.80729]
[6.1841, 6.73921]
[6.12226, 6.67182]
[6.06104, 6.6051]
[6.00043, 6.53905]
[5.94042, 6.47366]
[5.88102, 6.40893]
[5.82221, 6.34484]
[5.76399, 6.28139]
[5.70635, 6.21857]
[5.64928, 6.15639]
[5.59279, 6.09482]
[5.53686, 6.03388]
[5.48149, 5.97354]
[5.42668, 5.9138]
[5.37241, 5.85466]
[5.31869, 5.79612]
[5.2655, 5.73816]
[5.21285, 5.68077]
[5.16072, 5.62397]
[5.10911, 5.56773]
[5.05802, 5.51205]
[5.00744, 5.45693]
[4.95736, 5.40236]
[4.90779, 5.34834]
[4.85871, 5.2948

2-element Array{Float64,1}:
 0.47653155344276066
 0.5197486185099054 